In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
import numpy as np
import matplotlib.pyplot as plt

def create_series(df, xcol, datecol):
    # Create a dataframe with the features and the date time as the index
    features_considered = [xcol]
    features = df[features_considered]
    features.index = df[datecol]
    # features.head()
    # features.plot(subplots=True)
    return features


# X is the series to test
# log_x asks whether to log X prior to testing or not
def stationarity_test(X, log_x = "Y", return_p = False, print_res = True):
    
    # If X isn't logged, we need to log it for better results
    if log_x == "Y":
        X = np.log(X[X>0])
    
    # Once we have the series as needed we can do the ADF test
    from statsmodels.tsa.stattools import adfuller
    dickey_fuller = adfuller(X)
    
    if print_res:
    # If ADF statistic is < our 1% critical value (sig level) we can conclude it's not a fluke (ie low P val / reject H(0))
        print('ADF Stat is: {}.'.format(dickey_fuller[0]))
        # A lower p val means we can reject the H(0) that our data is NOT stationary
        print('P Val is: {}.'.format(dickey_fuller[1]))
        print('Critical Values (Significance Levels): ')
        for key,val in dickey_fuller[4].items():
            print(key,":",round(val,3))
            
    if return_p:
        return dickey_fuller[1]
    
# Differencing the data    
def difference(X):
    diff = X.diff()
    plt.plot(diff)
    plt.show()
    return diff

In [5]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
temp = df[['temp']]

In [38]:
temp.shape

(19706, 1)

In [28]:
if stationarity_test(temp, return_p=True, print_res = False) > 0.05:
    print("P Value is high. Consider Differencing: " + str(stationarity_test(temp, return_p = True, print_res = False)))
else:
   stationarity_test(temp)

P Value is high. Consider Differencing: 0.13207078999903976


In [39]:
from sklearn.preprocessing import MinMaxScaler
values = temp.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [41]:
scaled.shape

(19706, 1)

In [42]:
scaled = scaled.reshape((19706,))

In [43]:
scaled.shape

(19706,)

In [44]:
scaled

array([0.22170903, 0.19630487, 0.20554273, ..., 0.4618938 , 0.41801387,
       0.41801387], dtype=float32)

In [54]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [58]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 1
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [59]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [60]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

score = model.evaluate(X_test, y_test, verbose = 1)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)                    │ (None, 166, 64)             │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 10624)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │          10,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,881 (42.50 KB)

 Trainable params: 10,881 (42.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0242 - root_mean_squared_error: 0.1369 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0399 - learning_rate: 0.0010
Epoch 2/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0012 - root_mean_squared_error: 0.0340 - val_loss: 6.2294e-04 - val_root_mean_squared_error: 0.0250 - learning_rate: 0.0010
Epoch 3/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 6.6233e-04 - root_mean_squared_error: 0.0257 - val_loss: 5.5310e-04 - val_root_mean_squared_error: 0.0235 - learning_rate: 0.0010
Epoch 4/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4.9457e-04 - root_mean_squared_error: 0.0222 - val_loss: 3.8817e-04 - val_root_mean_squared_error: 0.0197 - learning_rate: 0.0010
Epoch 5/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 4.7907e-04 - root_mean_squared_error: 0.0219 - val_loss: 3.2330e-04 - val_root_mean_squared_error: 0.0180 - learning_rate: 0.0010
Epoch 6/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4.

In [61]:
forecast = model.predict(X_test)

123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [62]:
poll = np.array(df["temp"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [64]:
from sklearn.metrics import mean_squared_error

In [66]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 0.13477023
